In [1]:
# Import libraries
import warnings
!pip install kaggle
import kaggle
import pandas as pd
!pip install sqlalchemy

In [2]:
# Download dataset using kaggle api
!kaggle datasets download -d ankitbansal06/retail-orders

In [3]:
# Unzip the file
import zipfile
zip_ref = zipfile.ZipFile('retail-orders.zip')
zip_ref.extractall()
zip_ref.close()

In [4]:
#Load the Dataset into Jupyter Notebook and handling null values
data = pd.read_csv('orders.csv', na_values=['Not Available', 'unknown'])
data.head()

,Order Id,Order Date,Ship Mode,Segment,Country,City,State,Postal Code,Region,Category,Sub Category,Product Id,cost price,List Price,Quantity,Discount Percent
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5


In [5]:
# Rename columns names..make them lower case and replace space with unserscore
data.columns = data.columns.str.lower()
data.columns = data.columns.str.replace(" ", "_")
data.head()

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5


In [6]:
# Derive new columns discount, sale price and profit
data['discount'] = data['list_price']* data['discount_percent']*0.1
data['sale_price'] = data['list_price']-data['discount']
data['profit'] = data['sale_price']* data['cost_price']
data.head()

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent,discount,sale_price,profit
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2,52.0,208.0,49920.0
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3,219.0,511.0,306600.0
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5,5.0,5.0,50.0
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2,192.0,768.0,599040.0
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5,10.0,10.0,200.0


In [7]:
# Convert order date to date format
data['order_date'] = pd.to_datetime(data['order_date'], format = "%Y-%m-%d")

In [8]:
# Drop cost price, price and discount percent colums
data.drop(columns=["list_price", "cost_price", "discount_percent"], inplace=True)
data.head()

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,quantity,discount,sale_price,profit
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,2,52.0,208.0,49920.0
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,3,219.0,511.0,306600.0
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,2,5.0,5.0,50.0
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,5,192.0,768.0,599040.0
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,2,10.0,10.0,200.0


In [9]:
# Creating a connection to sql server database

from sqlalchemy import create_engine
engine = create_engine('mssql://DESKTOP-AFB98K7/master?driver=ODBC+DRIVER+17+FOR+SQL+SERVER')
conn = engine.connect()

In [10]:
# Loading the data into the database table
data.to_sql('orders', con = conn, index =False, if_exists = 'append')